In [6]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [7]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-330adb4f-c248-4289-94aa-4a032695ae97'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [8]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [9]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match("EUROPE", matchId)
    if match_detail["info"]["mapId"] in [1, 2, 11]:
        match = {}
        match["matchId"] = matchId
        match["gameDuration"] = match_detail["info"]["gameDuration"]
        match["gameType"] = match_detail["info"]["gameType"]
        match["gameVersion"] = match_detail["info"]["gameVersion"]
        matches.append(match)
        for index, row in enumerate(match_detail["info"]["participants"]):
            participants_row = {}
            participants_row["matchId"] = matchId
            participants_row["participant"] = index + 1
            participants_row["summonerName"] = row["summonerName"]
            participants_row["teamPosition"] = row["teamPosition"]
            participants_row["championName"] = row["championName"]
            participants_row["kda"] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
            participants_row["champLevel"] = row["champLevel"]
            participants_row["totalDamageToChampions"] = row[
                "totalDamageDealtToChampions"
            ]
            participants_row["totalDamageTaken"] = row["totalDamageTaken"]
            participants_row["totalHeal"] = row["totalHeal"]
            participants_row["totalMinionsKilled"] = row["totalMinionsKilled"]
            participants_row["goldEarned"] = row["goldEarned"]
            participants_row["summoner1"] = row["summoner1Id"]
            participants_row["summoner2"] = row["summoner2Id"]
            participants_row["item0"] = row["item0"]
            participants_row["item1"] = row["item1"]
            participants_row["item2"] = row["item2"]
            participants_row["item3"] = row["item3"]
            participants_row["item4"] = row["item4"]
            participants_row["item5"] = row["item5"]
            participants_row["item6"] = row["item6"]
            participants_row["win"] = row["win"]
            participants.append(participants_row)

        for row in timeline["info"]["frames"]:
            # frameCounter = 1
            # frame = row["{:02d}".format(frameCounter)]
            # frameCounter += 1
            for frameRow in row["events"]:
                timeline_row = {}
                timeline_row["matchId"] = matchId
                eventCounter = 0
                # eventFrame = frameRow["{:02d}".format(eventCounter)]
                eventFrame = frameRow
                eventCounter += 1

                if eventFrame["type"] == "CHAMPION_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["victimId"] = eventFrame["victimId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    if "bounty" in eventFrame:
                        timeline_row["bounty"] = eventFrame["bounty"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "BUILDING_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["teamId"]
                    timeline_row["buildingType"] = eventFrame["buildingType"]
                    timeline_row["laneType"] = eventFrame["laneType"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ITEM_PURCHASED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["itemId"] = eventFrame["itemId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_PLACED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["creatorId"] = eventFrame["creatorId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "LEVEL_UP":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["level"] = eventFrame["level"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ELITE_MONSTER_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["killerTeamId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    timeline_row["monsterType"] = eventFrame["monsterType"]
                    matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0)


In [10]:
match_detail["info"]["mapId"]

12

In [11]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5812018691,1,To Koi Ni,TOP,Malphite,0/12/8,15,10701,24804,818,...,4,12,3076,6662,3110,3047,1054,1011,3340,False
1,EUW1_5812018691,2,Niklaaaaaas,JUNGLE,Mordekaiser,13/5/6,17,28283,39449,10526,...,6,11,4633,3116,3157,3115,4637,3047,3340,False
2,EUW1_5812018691,3,Lethiferous Duck,BOTTOM,Senna,5/6/7,15,10063,16276,9604,...,4,21,3006,6672,3094,3042,1018,1038,3363,False
3,EUW1_5812018691,4,mi4u,MIDDLE,Zoe,11/9/14,16,30013,24562,2975,...,4,14,6655,3020,3157,3041,4628,3102,3363,False
4,EUW1_5812018691,5,Zokorá,UTILITY,Xerath,7/5/7,15,20423,14278,2082,...,4,7,3020,6655,3089,4628,0,0,3363,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,EUW1_5802111424,6,five night fredy,TOP,Darius,17/11/9,18,62453,78513,15489,...,4,6,3075,3078,6333,4401,3026,3111,3363,False
146,EUW1_5802111424,7,svagtljus,JUNGLE,FiddleSticks,11/13/18,18,41917,72274,26055,...,4,11,3152,3157,3020,3116,3135,3165,3330,False
147,EUW1_5802111424,8,Kelody,MIDDLE,Kaisa,14/19/15,18,59492,50008,3295,...,4,3,3042,3020,3157,3135,6655,4645,3363,False
148,EUW1_5802111424,9,SKTimo,BOTTOM,Lucian,18/13/13,18,54963,47371,3740,...,3,4,6671,3072,6694,3031,3047,3508,3363,False


In [12]:
matchTimelineDf["assistingParticipantIds"] = [ [] if x is np.NaN else x for x in matchTimelineDf["assistingParticipantIds"]]

In [13]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def checkAssist(row, name):
    matchId = row["matchId"]
    if row["assistingParticipantIds"]:
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0] == name:
                return True
    return False


In [14]:
# matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)

0       False
1       False
2       False
3       False
4       False
        ...  
9514    False
9515    False
9516    False
9517    False
9518    False
Length: 9519, dtype: bool

In [15]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)
matchTimelineDf["kelodyAssist"] = matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)


In [16]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5812018691,ITEM_PURCHASED,4.0,2033.0,2605,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,mi4u,Zoe,NaN,NaN,NaN,NaN,False
1,EUW1_5812018691,ITEM_PURCHASED,4.0,3340.0,3376,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,mi4u,Zoe,NaN,NaN,NaN,NaN,False
2,EUW1_5812018691,ITEM_PURCHASED,5.0,3850.0,4077,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Zokorá,Xerath,NaN,NaN,NaN,NaN,False
3,EUW1_5812018691,ITEM_PURCHASED,5.0,2003.0,4077,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Zokorá,Xerath,NaN,NaN,NaN,NaN,False
4,EUW1_5812018691,ITEM_PURCHASED,5.0,2003.0,4077,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Zokorá,Xerath,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9514,EUW1_5802111424,BUILDING_KILL,NaN,NaN,3012052,NaN,NaN,NaN,5.0,NaN,...,INHIBITOR_BUILDING,MID_LANE,50.0,NaN,NaN,Kil Jäger,Morgana,NaN,NaN,False
9515,EUW1_5802111424,ITEM_PURCHASED,8.0,2139.0,3018429,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,50.0,Kelody,Kaisa,NaN,NaN,NaN,NaN,False
9516,EUW1_5802111424,BUILDING_KILL,NaN,NaN,3020181,NaN,NaN,NaN,1.0,NaN,...,TOWER_BUILDING,MID_LANE,50.0,NaN,NaN,H4rdSlayer,Aatrox,NaN,NaN,False
9517,EUW1_5802111424,BUILDING_KILL,NaN,NaN,3023352,NaN,NaN,NaN,3.0,NaN,...,TOWER_BUILDING,MID_LANE,50.0,NaN,NaN,Portzas,Ryze,NaN,NaN,False


In [17]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
7,EUW1_5812018691,8
17,EUW1_5812033705,8
26,EUW1_5809632920,7
30,EUW1_5808289239,1
46,EUW1_5808285447,7
51,EUW1_5808262634,2
66,EUW1_5803852070,7
76,EUW1_5803774910,7
81,EUW1_5803660385,2
91,EUW1_5803299160,2


In [18]:
kelodyEvents = []
#TODO bool if kelody was assist or not
# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5812018691,ITEM_PURCHASED,8.0,2033.0,18597,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Vex,NaN,NaN,NaN,NaN,False
1,EUW1_5812018691,ITEM_PURCHASED,8.0,3340.0,19298,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Vex,NaN,NaN,NaN,NaN,False
2,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,117994,NaN,NaN,NaN,4.0,8.0,...,NaN,NaN,2.0,NaN,NaN,mi4u,Zoe,Kelody,Vex,False
3,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,119600,NaN,NaN,NaN,8.0,4.0,...,NaN,NaN,2.0,NaN,NaN,Kelody,Vex,mi4u,Zoe,False
4,EUW1_5812018691,LEVEL_UP,8.0,NaN,157621,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,Kelody,Vex,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,EUW1_5802111424,WARD_PLACED,NaN,NaN,2933102,NaN,BLUE_TRINKET,8.0,NaN,NaN,...,NaN,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,False
1280,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,2948844,NaN,NaN,NaN,8.0,4.0,...,NaN,NaN,49.0,NaN,NaN,Kelody,Kaisa,skxfox,Ashe,False
1281,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,2952557,NaN,NaN,NaN,1.0,8.0,...,NaN,NaN,49.0,NaN,NaN,H4rdSlayer,Aatrox,Kelody,Kaisa,False
1282,EUW1_5802111424,ITEM_PURCHASED,8.0,2139.0,2962531,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,49.0,Kelody,Kaisa,NaN,NaN,NaN,NaN,False


In [19]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == True]

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
11,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,321079,NaN,NaN,NaN,7.0,4.0,...,NaN,NaN,5.0,NaN,NaN,ARRAH Jajajan,Shaco,mi4u,Zoe,True
14,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,390266,NaN,NaN,NaN,7.0,4.0,...,NaN,NaN,7.0,NaN,NaN,ARRAH Jajajan,Shaco,mi4u,Zoe,True
38,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,1045209,NaN,NaN,NaN,7.0,3.0,...,NaN,NaN,17.0,NaN,NaN,ARRAH Jajajan,Shaco,Lethiferous Duck,Senna,True
40,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,1059520,NaN,NaN,NaN,6.0,2.0,...,NaN,NaN,18.0,NaN,NaN,InRevolt,Gangplank,Niklaaaaaas,Mordekaiser,True
42,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,1065766,NaN,NaN,NaN,6.0,4.0,...,NaN,NaN,18.0,NaN,NaN,InRevolt,Gangplank,mi4u,Zoe,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,2458664,NaN,NaN,NaN,9.0,2.0,...,NaN,NaN,41.0,NaN,NaN,SKTimo,Lucian,Βuzzy,Rengar,True
1265,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,2573745,NaN,NaN,NaN,7.0,1.0,...,NaN,NaN,43.0,NaN,NaN,svagtljus,FiddleSticks,H4rdSlayer,Aatrox,True
1269,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,2694423,NaN,NaN,NaN,9.0,4.0,...,NaN,NaN,45.0,NaN,NaN,SKTimo,Lucian,skxfox,Ashe,True
1270,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,2698485,NaN,NaN,NaN,9.0,5.0,...,NaN,NaN,45.0,NaN,NaN,SKTimo,Lucian,Kil Jäger,Morgana,True


Check how often a Player died in Early Game (pre 10 mins)

In [20]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
7,EUW1_5812018691,8,3
17,EUW1_5812033705,8,1
26,EUW1_5809632920,7,1
30,EUW1_5808289239,1,4
46,EUW1_5808285447,7,1
51,EUW1_5808262634,2,2
66,EUW1_5803852070,7,1
76,EUW1_5803774910,7,1
81,EUW1_5803660385,2,2
91,EUW1_5803299160,2,1


#### TODO
##### Features:
- <s>Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)</s>

- Died with ... Gold

- <s>count 1v1 kill/deaths</s>  
  
- Good roaming (assists pre 15 min)

- Warding (check ward placements pre 15 min) maybe in combination with ganks

- CS per min (if data available)

- KDA (kills and assists divided by deaths) als Output KDA String aus participantDf

- <s>Tower Kills/Assists</s> 

- <s>Objective Kills/Assists (Baron, Dragon, Herald)</s> 



##### Features on the tableau visualization:

- <s>Timeline</s>

- <s>Champion Icons</s>

- <s>Damage Dealt Bar Chart (participantDf)</s>

- Game win (participantDf)



Check 1v1

In [21]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == False]

def check1v1(row, name):
    if not row["assistingParticipantIds"] and (row["victimName"] == name or row["killerName"] == name) and (row["type"] == "CHAMPION_KILL"):
        return True
    return False

kelodyEventsDf["1v1"] = kelodyEventsDf.apply(lambda row: check1v1(row, "Kelody"), axis = 1)
kelodyEventsDf[kelodyEventsDf["1v1"] == True]

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist,1v1
2,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,117994,NaN,NaN,NaN,4.0,8.0,...,NaN,2.0,NaN,NaN,mi4u,Zoe,Kelody,Vex,False,True
10,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,316573,NaN,NaN,NaN,4.0,8.0,...,NaN,5.0,NaN,NaN,mi4u,Zoe,Kelody,Vex,False,True
17,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,538556,NaN,NaN,NaN,4.0,8.0,...,NaN,9.0,NaN,NaN,mi4u,Zoe,Kelody,Vex,False,True
32,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,973436,NaN,NaN,NaN,8.0,1.0,...,NaN,16.0,NaN,NaN,Kelody,Vex,To Koi Ni,Malphite,False,True
62,EUW1_5812018691,CHAMPION_KILL,NaN,NaN,1553694,NaN,NaN,NaN,8.0,5.0,...,NaN,26.0,NaN,NaN,Kelody,Vex,Zokorá,Xerath,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,1204386,NaN,NaN,NaN,2.0,8.0,...,NaN,20.0,NaN,NaN,Βuzzy,Rengar,Kelody,Kaisa,False,True
1220,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,1596805,NaN,NaN,NaN,2.0,8.0,...,NaN,27.0,NaN,NaN,Βuzzy,Rengar,Kelody,Kaisa,False,True
1237,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,1986881,NaN,NaN,NaN,2.0,8.0,...,NaN,33.0,NaN,NaN,Βuzzy,Rengar,Kelody,Kaisa,False,True
1252,EUW1_5802111424,CHAMPION_KILL,NaN,NaN,2307378,NaN,NaN,NaN,4.0,8.0,...,NaN,38.0,NaN,NaN,skxfox,Ashe,Kelody,Kaisa,False,True


In [22]:
def count1v1(row, name, killer):
  if killer:
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["killerName"] == name)])
  else: 
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["victimName"] == name)])

kelody["1v1Won"] = kelody.apply(lambda row: count1v1(row, "Kelody", True), axis=1)
kelody["1v1Lost"] = kelody.apply(lambda row: count1v1(row, "Kelody", False), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost
7,EUW1_5812018691,8,3,2,4
17,EUW1_5812033705,8,1,5,2
26,EUW1_5809632920,7,1,3,0
30,EUW1_5808289239,1,4,2,7
46,EUW1_5808285447,7,1,0,1
51,EUW1_5808262634,2,2,2,2
66,EUW1_5803852070,7,1,11,0
76,EUW1_5803774910,7,1,7,1
81,EUW1_5803660385,2,2,5,1
91,EUW1_5803299160,2,1,2,0


In [23]:
def countTurretKills(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "BUILDING_KILL") & (kelodyEventsDf["killerName"] == name) & (kelodyEventsDf["1v1"] == False)])

kelody["turretKills"] = kelody.apply(lambda row: countTurretKills(row, "Kelody"), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills
7,EUW1_5812018691,8,3,2,4,0
17,EUW1_5812033705,8,1,5,2,2
26,EUW1_5809632920,7,1,3,0,0
30,EUW1_5808289239,1,4,2,7,1
46,EUW1_5808285447,7,1,0,1,0
51,EUW1_5808262634,2,2,2,2,1
66,EUW1_5803852070,7,1,11,0,1
76,EUW1_5803774910,7,1,7,1,0
81,EUW1_5803660385,2,2,5,1,0
91,EUW1_5803299160,2,1,2,0,6


In [24]:
def counEliteMonsterKills(row, monsterName):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "ELITE_MONSTER_KILL") & (kelodyEventsDf["monsterType"] == monsterName)])

kelody["dragonKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "DRAGON"), axis=1)
kelody["baronKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "BARON_NASHOR"), axis=1)
kelody["heraldKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "RIFTHERALD"), axis=1)

kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills
7,EUW1_5812018691,8,3,2,4,0,1,0,0
17,EUW1_5812033705,8,1,5,2,2,0,0,0
26,EUW1_5809632920,7,1,3,0,0,3,0,1
30,EUW1_5808289239,1,4,2,7,1,0,0,0
46,EUW1_5808285447,7,1,0,1,0,0,0,0
51,EUW1_5808262634,2,2,2,2,1,2,0,1
66,EUW1_5803852070,7,1,11,0,1,3,0,1
76,EUW1_5803774910,7,1,7,1,0,2,0,1
81,EUW1_5803660385,2,2,5,1,0,3,0,0
91,EUW1_5803299160,2,1,2,0,6,1,0,2


In [25]:
def deathToGank(row, name):
    if row["1v1"] == False and row["type"] == "CHAMPION_KILL" and row["victimName"] == name: 
        enemyJungler = participantsDf.loc[(row["matchId"] == participantsDf["matchId"]) & (participantsDf["teamPosition"] == "JUNGLE")]["summonerName"].values[0]
        if row["killerName"] == enemyJungler:
            return True
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == row["matchId"])]["teamPosition"].values[0] == "JUNGLE":
                return True
    return False
kelodyEventsDf["deathToGank"] = kelodyEventsDf.apply(lambda row: deathToGank(row, "Kelody"), axis=1)

In [26]:
def countDeathsToGanks(row, name):
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["deathToGank"] == True)
            & (kelodyEventsDf["victimName"] == name)
            & (kelodyEventsDf["timestampMIN"] <= 15)
        ]
    )


kelody["deathsToGanks"] = kelody.apply(
    lambda row: countDeathsToGanks(row, "Kelody"), axis=1
)


In [27]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks
7,EUW1_5812018691,8,3,2,4,0,1,0,0,0
17,EUW1_5812033705,8,1,5,2,2,0,0,0,0
26,EUW1_5809632920,7,1,3,0,0,3,0,1,1
30,EUW1_5808289239,1,4,2,7,1,0,0,0,0
46,EUW1_5808285447,7,1,0,1,0,0,0,0,1
51,EUW1_5808262634,2,2,2,2,1,2,0,1,0
66,EUW1_5803852070,7,1,11,0,1,3,0,1,1
76,EUW1_5803774910,7,1,7,1,0,2,0,1,0
81,EUW1_5803660385,2,2,5,1,0,3,0,0,0
91,EUW1_5803299160,2,1,2,0,6,1,0,2,1


In [28]:
participantsDf


,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5812018691,1,To Koi Ni,TOP,Malphite,0/12/8,15,10701,24804,818,...,4,12,3076,6662,3110,3047,1054,1011,3340,False
1,EUW1_5812018691,2,Niklaaaaaas,JUNGLE,Mordekaiser,13/5/6,17,28283,39449,10526,...,6,11,4633,3116,3157,3115,4637,3047,3340,False
2,EUW1_5812018691,3,Lethiferous Duck,BOTTOM,Senna,5/6/7,15,10063,16276,9604,...,4,21,3006,6672,3094,3042,1018,1038,3363,False
3,EUW1_5812018691,4,mi4u,MIDDLE,Zoe,11/9/14,16,30013,24562,2975,...,4,14,6655,3020,3157,3041,4628,3102,3363,False
4,EUW1_5812018691,5,Zokorá,UTILITY,Xerath,7/5/7,15,20423,14278,2082,...,4,7,3020,6655,3089,4628,0,0,3363,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,EUW1_5802111424,6,five night fredy,TOP,Darius,17/11/9,18,62453,78513,15489,...,4,6,3075,3078,6333,4401,3026,3111,3363,False
146,EUW1_5802111424,7,svagtljus,JUNGLE,FiddleSticks,11/13/18,18,41917,72274,26055,...,4,11,3152,3157,3020,3116,3135,3165,3330,False
147,EUW1_5802111424,8,Kelody,MIDDLE,Kaisa,14/19/15,18,59492,50008,3295,...,4,3,3042,3020,3157,3135,6655,4645,3363,False
148,EUW1_5802111424,9,SKTimo,BOTTOM,Lucian,18/13/13,18,54963,47371,3740,...,3,4,6671,3072,6694,3031,3047,3508,3363,False


In [29]:
def countWardsPlaced(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["participant"].values[0]
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["creatorId"] == participantId)
            & (kelodyEventsDf["type"] == "WARD_PLACED")
        ]
    )


kelody["wardsPlaced"] = kelody.apply(
    lambda row: countWardsPlaced(row, "Kelody"), axis=1
)


In [30]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced
7,EUW1_5812018691,8,3,2,4,0,1,0,0,0,19
17,EUW1_5812033705,8,1,5,2,2,0,0,0,0,10
26,EUW1_5809632920,7,1,3,0,0,3,0,1,1,8
30,EUW1_5808289239,1,4,2,7,1,0,0,0,0,9
46,EUW1_5808285447,7,1,0,1,0,0,0,0,1,3
51,EUW1_5808262634,2,2,2,2,1,2,0,1,0,5
66,EUW1_5803852070,7,1,11,0,1,3,0,1,1,5
76,EUW1_5803774910,7,1,7,1,0,2,0,1,0,7
81,EUW1_5803660385,2,2,5,1,0,3,0,0,0,4
91,EUW1_5803299160,2,1,2,0,6,1,0,2,1,11


In [40]:
def countWardsPlacedBeforeGanked(row, name):
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["deathToGank"] == True)
            & (kelodyEventsDf["victimName"] == name)
        ]
    )

    # TODO


# deaths to gank
def getTimesWardsPlacedBeforeGank(row):
    return kelodyEventsDf[
        (kelodyEventsDf["matchId"] == row["matchId"])
        & (kelodyEventsDf["deathToGank"] == True)
        & (kelodyEventsDf["timestampMIN"] <= 15)
    ]["timestampMIN"]


kelodyEventsDf[
    (kelodyEventsDf["deathToGank"] == True) & (kelodyEventsDf["timestampMIN"] <= 15)
]["timestampMIN"]




23      12.0
165      7.0
178     11.0
181     13.0
186     14.0
327      9.0
338     13.0
343     15.0
472      8.0
568     10.0
718      3.0
1087     8.0
1097    10.0
1117    14.0
Name: timestampMIN, dtype: float64

In [32]:
def checkRoamKills(row, name):
    if row["assistingParticipantIds"] and (row["killerName"] == name) and (row["type"] == "CHAMPION_KILL") and (row["timestampMIN"] <= 15):
        return True
    return False

def countRoamKills(row):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & ((kelodyEventsDf["kelodyAssist"] == True) & (kelodyEventsDf["timestampMIN"] <= 15) | (kelodyEventsDf["kelodyRoamKill"] == True))])

In [33]:
kelodyEventsDf["kelodyRoamKill"] = kelodyEventsDf.apply(lambda row: checkRoamKills(row, "Kelody"), axis = 1)
kelody["roamKillsAssists"] = kelody.apply(lambda row: countRoamKills(row), axis=1)

In [34]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced,roamKillsAssists
7,EUW1_5812018691,8,3,2,4,0,1,0,0,0,19,3
17,EUW1_5812033705,8,1,5,2,2,0,0,0,0,10,3
26,EUW1_5809632920,7,1,3,0,0,3,0,1,1,8,4
30,EUW1_5808289239,1,4,2,7,1,0,0,0,0,9,0
46,EUW1_5808285447,7,1,0,1,0,0,0,0,1,3,6
51,EUW1_5808262634,2,2,2,2,1,2,0,1,0,5,8
66,EUW1_5803852070,7,1,11,0,1,3,0,1,1,5,6
76,EUW1_5803774910,7,1,7,1,0,2,0,1,0,7,5
81,EUW1_5803660385,2,2,5,1,0,3,0,0,0,4,9
91,EUW1_5803299160,2,1,2,0,6,1,0,2,1,11,9


In [35]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
